In [13]:
import constants, Plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
myCmap=sns.diverging_palette(170, 330, l=65, center="dark", as_cmap=True)
physical = {constants.SHOCK: ['Time', 'Density', 'gasTemp', 'av', 'zeta', 'radfield', constants.SHOCKVEL],
            constants.HOTCORE: ['Time', 'Density', 'gasTemp', 'av', 'zeta', 'radfield']}
species=['#CH3OH', 'CH3OH', '#SIO', 'SIO']

In [15]:
def buildDataframe(tipo): 
    df= pd.read_csv(tipo+'.csv', index_col=0)

    df = df.loc[:,physical[tipo]+species+['runName']]
    for prop in physical[tipo]+species:
        with np.errstate(divide='ignore'): df[f'{prop}_log']=np.log10(df[prop])
    
    return df

In [16]:
def getCorrMatrix(df, tipo, singleAxis):
    xaxis=[f'{prop}_log' for prop in species]
    yaxis= xaxis if singleAxis else [f'{prop}_log' for prop in physical[tipo]]

    cor = df.loc[:,xaxis if singleAxis else xaxis+yaxis].corr()
    cor=cor[cor.abs().ge(0.5)].loc[xaxis,yaxis].dropna(how='all').dropna(how='all', axis=1)
    return cor

In [17]:
def isValid(x, y):
    phases=['#','@','$']
    if x==y: return False
    elif x.strip('#@$')==y.strip('#@$'): return True
    for sym in phases:
        if sym in x and sym in y: return True
    return False

def getCorrValues(df, tipo, corrList, singleAxis):
    cor=getCorrMatrix(df, tipo, singleAxis)

    for i in cor.index:
        for j in cor.columns:
            if not np.isnan(cor.loc[i,j]):
                x=j.replace('_log','')
                y=i.replace('_log','')
                if singleAxis:
                    if isValid(x,y): corrList.append(pd.DataFrame(data={'tipo': tipo, 'x': x, 'y': y, 'coef': cor.loc[i,j]}, index=[0]))
                else: corrList.append(pd.DataFrame(data={'tipo': tipo, 'x': x, 'y': y, 'coef': cor.loc[i,j]}, index=[0]))

In [18]:
corrList=[]
for tipo in physical:
    df=buildDataframe(tipo)
    for singleAxis in [False, True]:
        getCorrValues(df, tipo, corrList, singleAxis)

corrs=pd.concat(corrList, ignore_index=True)

In [19]:
corrs

,tipo,x,y,coef
0,c shock,gasTemp,#CH3OH,-0.528318
1,c shock,SIO,#SIO,0.500797
2,c shock,#SIO,SIO,0.500797
3,hot core,Time,#CH3OH,-0.560787
4,hot core,Time,CH3OH,0.531724
5,hot core,radfield,#SIO,-0.554428
6,hot core,Time,SIO,0.733332
7,hot core,gasTemp,SIO,0.675674
